In [1]:
import pandas as pd
import os
import glob
import numpy as np

# Attributes Analysis

In [2]:
pd.set_option('display.max_colwidth', 1000)

In [3]:
all_data_path = '/Users/sonynka/HTW/IC/data/'
df = pd.read_csv(os.path.join(all_data_path, 'aboutyou.csv'), sep=';', encoding='utf-8')

## Tags
Analyse most used tags

In [4]:
df.describe()

,img_path,img_url,category,subcategory,color,product_name,attributes
count,20776,20776,20776,20776,20776,20776,20776
unique,20079,16640,8,66,9,11105,16197
top,/Users/sonynka/HTW/IC/data/aboutyou/kleider/etuikleider/e97b08b47ccad8026a85b07a3eaafafa.jpg,https://cdn.aboutstatic.com/file/71cb3838189b078bb9f44dd5e017ca83,kleider,feinstrickpullover,blue,Ashley Brooke by heine | Cocktailkleid,"Stehkragen,Taillierter Schnitt,Seitliche Eingrifftaschen,Unifarben,Steppjacke,Daune,Teilgefüttert,Label Patch/Label Flag,Ton-in-Ton-Nähte,Zusammensetzung: 100 % Polyamid,Elastizität: Elastisch/stretch,Passform: Normale Passform"
freq,3,6,5429,821,4256,46,22


In [5]:
df['tags'] = df['attributes'].apply(lambda x: x.lower().split(','))

In [6]:
tag_count = {}
def count_tags(tags):
    for tag in tags:
        t = tag.lower()
        if t in tag_count:
            tag_count[t] += 1
        else:
            tag_count[t] = 1
            
a = df['tags'].apply(count_tags)

In [7]:
tag_df = pd.Series(tag_count).reset_index().rename(columns={'index': 'tag', 0: 'count'})
num_imgs = df.shape[0]
tag_df['ratio'] = (pd.to_numeric(tag_df['count']) / num_imgs * 100).round(2)
tag_df = tag_df[tag_df['ratio'] > 1]
tag_df = tag_df.sort_values('ratio', ascending=False)

In [8]:
tag_df['tag'].unique().tolist()

['passform: normale passform',
 'unifarben',
 'elastizität: nicht elastisch',
 'weicher griff',
 'elastizität: leicht elastisch',
 'länge: normale länge',
 'ärmellänge: langarm',
 'rundhals-ausschnitt',
 'länge: lang/maxi',
 'all-over-muster',
 'seitliche eingrifftaschen',
 'obermaterial: 100 % polyester',
 'länge: kurz/mini',
 'baumwolle',
 'ärmellänge: ärmellos',
 'länge: knielang',
 'taillierter schnitt',
 'leibhöhe: mid waist',
 'kragenlos',
 'glatter stoff',
 'reißverschluss',
 'elastizität: elastisch/stretch',
 'viskose',
 'fließender stoff',
 'futter: 100 % polyester',
 'ton-in-ton-nähte',
 'abgesteppter saum/kante',
 'spitze',
 'elastischer bund/saum',
 ' 5 % elasthan',
 'stehkragen',
 'blusenshirt',
 'gürtelschlaufen',
 'label patch/label flag',
 'leicht transparent',
 'gerader saum',
 'jeans',
 'gerader schnitt',
 'knopfleiste',
 'ärmellänge: viertelarm',
 'geblümt/floral',
 'rücken-reißverschluss',
 'v-ausschnitt',
 'ärmellänge: dreiviertelarm',
 'jersey',
 'obermaterial: 10

## Select useful attributes
Find attributes that are used a lot and can be useful for the application.
Create a special column in the dataframe with each of the useful tags.

In [9]:
def create_tag_column(df, tag_list, column_name):
    """ 
        Create a new column in the given dataframe with the specified column name. 
        The column holds the tag if the tag is in the tag_list otherwise it holds np.nan.
    """
    
    df[column_name] = df['tags'].apply(lambda x: [s for s in x if s in tag_list])
    df[column_name] = [x[0].split(':')[-1].strip() if len(x) > 0 else np.nan for x in df[column_name]]
    
    return df

### Ärmellänge

In [10]:
tag_df[tag_df['tag'].str.startswith('ärmellänge')]

,tag,count,ratio
3412,ärmellänge: langarm,4654,22.40
3414,ärmellänge: ärmellos,2779,13.38
3413,ärmellänge: viertelarm,1642,7.90
3410,ärmellänge: dreiviertelarm,1536,7.39
3411,ärmellänge: halbarm,393,1.89


In [11]:
aermel_list = tag_df[tag_df['tag'].str.startswith('ärmellänge')]['tag'].tolist()
df = create_tag_column(df, aermel_list, 'ärmellänge')

In [12]:
df['ärmellänge'].unique()

array([nan, 'viertelarm', 'ärmellos', 'halbarm', 'dreiviertelarm',
       'langarm'], dtype=object)

### Länge

In [13]:
tag_df[tag_df['tag'].str.startswith('länge')]

,tag,count,ratio
1589,länge: normale länge,5089,24.49
1587,länge: lang/maxi,3729,17.95
1585,länge: kurz/mini,2991,14.40
1584,länge: knielang,2720,13.09
1583,länge: 7/8-lang,753,3.62
1588,länge: langer schnitt,501,2.41
1582,länge: 3/4-lang,244,1.17


In [14]:
laenge_list = tag_df[tag_df['tag'].str.startswith('länge')]['tag'].tolist()
df = create_tag_column(df, laenge_list, 'länge')

In [15]:
df['länge'].unique()

array(['kurz/mini', 'knielang', '7/8-lang', '3/4-lang', 'lang/maxi', nan,
       'normale länge', 'langer schnitt'], dtype=object)

### Ausschnitt

In [16]:
tag_df[tag_df['tag'].str.contains('ausschnitt')]

,tag,count,ratio
2797,rundhals-ausschnitt,3945,18.99
3044,v-ausschnitt,1544,7.43
2801,rückenausschnitt,536,2.58
2989,tiefer ausschnitt/dekolleté,523,2.52
1062,eingefasster ausschnitt,341,1.64
2999,tunika-ausschnitt,294,1.42
3009,u-boot-ausschnitt,265,1.28


In [17]:
ausschnitt_list = tag_df[tag_df['tag'].str.contains('ausschnitt')]['tag'].tolist()
df = create_tag_column(df, ausschnitt_list, 'ausschnitt')

In [18]:
df['ausschnitt'].unique()

array([nan, 'rundhals-ausschnitt', 'rückenausschnitt', 'v-ausschnitt',
       'tiefer ausschnitt/dekolleté', 'u-boot-ausschnitt',
       'eingefasster ausschnitt', 'tunika-ausschnitt'], dtype=object)

### Muster

In [19]:
tag_df[(tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze$'))]

,tag,count,ratio
864,all-over-muster,3369,16.22
2881,spitze,2035,9.79
1303,geblümt/floral,1562,7.52
1322,gestreift,1142,5.50
1313,gepunktet,393,1.89
2871,seitliche streifen,218,1.05


In [20]:
muster_list = tag_df[(tag_df['tag'].str.contains(r'floral|streif|punkt|muster|spitze$'))]['tag'].tolist()
df = create_tag_column(df, muster_list, 'muster')

In [21]:
df['muster'].unique()

array([nan, 'geblümt/floral', 'all-over-muster', 'gepunktet',
       'seitliche streifen', 'gestreift', 'spitze'], dtype=object)

### Passform

In [22]:
tag_df[(tag_df['tag'].str.contains('passform'))]

,tag,count,ratio
2720,passform: normale passform,13816,66.50
2718,passform: lockere passform,1317,6.34
2721,passform: regular,1273,6.13
2716,passform: figurbetonte passform,1213,5.84
2723,passform: skinny,802,3.86
2724,passform: slimfit,551,2.65
2722,passform: schmale passform,488,2.35
2719,passform: loosefit,343,1.65
2725,passform: tapered,272,1.31


In [23]:
passform_list = tag_df[(tag_df['tag'].str.contains('passform'))]['tag'].tolist()
df = create_tag_column(df, passform_list, 'passform')

In [24]:
df['passform'].unique()

array(['normale passform', 'schmale passform', 'figurbetonte passform',
       'lockere passform', nan, 'regular', 'tapered', 'slimfit',
       'loosefit', 'skinny'], dtype=object)

In [25]:
df = df.drop(['attributes', 'tags', 'product_name'], axis=1)
df.head()

,img_path,img_url,category,subcategory,color,ärmellänge,länge,ausschnitt,muster,passform
0,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/d4505b3e5dd14ae01d741436b73bf2c6.jpg,https://cdn.aboutstatic.com/file/d4505b3e5dd14ae01d741436b73bf2c6,röcke,jeansröcke,black,NaN,kurz/mini,NaN,NaN,normale passform
1,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/3cf403b37a8f7e48cc538fa6104004d5.jpg,https://cdn.aboutstatic.com/file/3cf403b37a8f7e48cc538fa6104004d5,röcke,jeansröcke,black,NaN,knielang,NaN,NaN,normale passform
2,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/4a6b1747736bd16a787881c1baf49f30.jpg,https://cdn.aboutstatic.com/file/4a6b1747736bd16a787881c1baf49f30,röcke,jeansröcke,black,NaN,kurz/mini,NaN,NaN,normale passform
3,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/627a34949581084df25783a9e726e393.jpg,https://cdn.aboutstatic.com/file/627a34949581084df25783a9e726e393,röcke,jeansröcke,black,NaN,kurz/mini,NaN,geblümt/floral,schmale passform
4,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/eefac4400c854f06740b75131a595a17.jpg,https://cdn.aboutstatic.com/file/eefac4400c854f06740b75131a595a17,röcke,jeansröcke,black,NaN,kurz/mini,NaN,NaN,normale passform


# Create dummies

In [26]:
df = df.set_index('img_url')
df.head()

,img_path,category,subcategory,color,ärmellänge,länge,ausschnitt,muster,passform
img_url,,,,,,,,,
https://cdn.aboutstatic.com/file/d4505b3e5dd14ae01d741436b73bf2c6,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/d4505b3e5dd14ae01d741436b73bf2c6.jpg,röcke,jeansröcke,black,NaN,kurz/mini,NaN,NaN,normale passform
https://cdn.aboutstatic.com/file/3cf403b37a8f7e48cc538fa6104004d5,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/3cf403b37a8f7e48cc538fa6104004d5.jpg,röcke,jeansröcke,black,NaN,knielang,NaN,NaN,normale passform
https://cdn.aboutstatic.com/file/4a6b1747736bd16a787881c1baf49f30,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/4a6b1747736bd16a787881c1baf49f30.jpg,röcke,jeansröcke,black,NaN,kurz/mini,NaN,NaN,normale passform
https://cdn.aboutstatic.com/file/627a34949581084df25783a9e726e393,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/627a34949581084df25783a9e726e393.jpg,röcke,jeansröcke,black,NaN,kurz/mini,NaN,geblümt/floral,schmale passform
https://cdn.aboutstatic.com/file/eefac4400c854f06740b75131a595a17,/Users/sonynka/HTW/IC/data/aboutyou/röcke/jeansröcke/eefac4400c854f06740b75131a595a17.jpg,röcke,jeansröcke,black,NaN,kurz/mini,NaN,NaN,normale passform


In [27]:
df_dum = pd.get_dummies(df[['category', 'color', 'ärmellänge', 'länge', 'muster', 'passform', 'ausschnitt']]).reset_index()

In [28]:
df_dum.head()

,img_url,category_blusen & tuniken,category_hosen,category_jacken,category_jeans,category_jumpsuits & overalls,category_kleider,category_röcke,category_strick,color_beige,...,passform_skinny,passform_slimfit,passform_tapered,ausschnitt_eingefasster ausschnitt,ausschnitt_rundhals-ausschnitt,ausschnitt_rückenausschnitt,ausschnitt_tiefer ausschnitt/dekolleté,ausschnitt_tunika-ausschnitt,ausschnitt_u-boot-ausschnitt,ausschnitt_v-ausschnitt
0,https://cdn.aboutstatic.com/file/d4505b3e5dd14ae01d741436b73bf2c6,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://cdn.aboutstatic.com/file/3cf403b37a8f7e48cc538fa6104004d5,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,https://cdn.aboutstatic.com/file/4a6b1747736bd16a787881c1baf49f30,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,https://cdn.aboutstatic.com/file/627a34949581084df25783a9e726e393,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,https://cdn.aboutstatic.com/file/eefac4400c854f06740b75131a595a17,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_dum = df_dum.groupby(['img_url']).max().reset_index()

In [30]:
df_dum.head()

,img_url,category_blusen & tuniken,category_hosen,category_jacken,category_jeans,category_jumpsuits & overalls,category_kleider,category_röcke,category_strick,color_beige,...,passform_skinny,passform_slimfit,passform_tapered,ausschnitt_eingefasster ausschnitt,ausschnitt_rundhals-ausschnitt,ausschnitt_rückenausschnitt,ausschnitt_tiefer ausschnitt/dekolleté,ausschnitt_tunika-ausschnitt,ausschnitt_u-boot-ausschnitt,ausschnitt_v-ausschnitt
0,https://cdn.aboutstatic.com/file/00005b236646a41122e2af6608f9bc74,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://cdn.aboutstatic.com/file/00170e6b9430ae87eb09a4f77df8a95c,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,https://cdn.aboutstatic.com/file/0019e41ae7bebcb785f8c7ebc13dc37b,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,https://cdn.aboutstatic.com/file/00238febf34d56cb2d196c0043d83131,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,https://cdn.aboutstatic.com/file/0026d05e1d0ef44fae56b761a1165c7b,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df_paths = df.groupby(['img_url']).first()['img_path'].reset_index()

In [32]:
df_paths.head()

,img_url,img_path
0,https://cdn.aboutstatic.com/file/00005b236646a41122e2af6608f9bc74,/Users/sonynka/HTW/IC/data/aboutyou/hosen/chinos/00005b236646a41122e2af6608f9bc74.jpg
1,https://cdn.aboutstatic.com/file/00170e6b9430ae87eb09a4f77df8a95c,/Users/sonynka/HTW/IC/data/aboutyou/jumpsuits & overalls/lange jumpsuits/00170e6b9430ae87eb09a4f77df8a95c.jpg
2,https://cdn.aboutstatic.com/file/0019e41ae7bebcb785f8c7ebc13dc37b,/Users/sonynka/HTW/IC/data/aboutyou/röcke/miniröcke/0019e41ae7bebcb785f8c7ebc13dc37b.jpg
3,https://cdn.aboutstatic.com/file/00238febf34d56cb2d196c0043d83131,/Users/sonynka/HTW/IC/data/aboutyou/kleider/minikleider/00238febf34d56cb2d196c0043d83131.jpg
4,https://cdn.aboutstatic.com/file/0026d05e1d0ef44fae56b761a1165c7b,/Users/sonynka/HTW/IC/data/aboutyou/hosen/shorts/0026d05e1d0ef44fae56b761a1165c7b.jpg


In [33]:
df_dum = df_paths.merge(df_dum, how='inner').drop(['img_url'], axis=1)

In [34]:
df_dum.head()

,img_path,category_blusen & tuniken,category_hosen,category_jacken,category_jeans,category_jumpsuits & overalls,category_kleider,category_röcke,category_strick,color_beige,...,passform_skinny,passform_slimfit,passform_tapered,ausschnitt_eingefasster ausschnitt,ausschnitt_rundhals-ausschnitt,ausschnitt_rückenausschnitt,ausschnitt_tiefer ausschnitt/dekolleté,ausschnitt_tunika-ausschnitt,ausschnitt_u-boot-ausschnitt,ausschnitt_v-ausschnitt
0,/Users/sonynka/HTW/IC/data/aboutyou/hosen/chinos/00005b236646a41122e2af6608f9bc74.jpg,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,/Users/sonynka/HTW/IC/data/aboutyou/jumpsuits & overalls/lange jumpsuits/00170e6b9430ae87eb09a4f77df8a95c.jpg,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,/Users/sonynka/HTW/IC/data/aboutyou/röcke/miniröcke/0019e41ae7bebcb785f8c7ebc13dc37b.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,/Users/sonynka/HTW/IC/data/aboutyou/kleider/minikleider/00238febf34d56cb2d196c0043d83131.jpg,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,/Users/sonynka/HTW/IC/data/aboutyou/hosen/shorts/0026d05e1d0ef44fae56b761a1165c7b.jpg,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
df_dum['img_path'] = df_dum['img_path'].apply(lambda x: x[35:])

In [36]:
df_dum.to_csv('/Users/sonynka/HTW/IC/data/img_attr.csv', sep='\t', encoding='utf-8', index=False)